# Sífilis adquirida

Baseado nos passos de limpeza e profiling feitos anteriormente, alguns serão reproduzidos para tratar e analisar os dados.


In [1]:
# imports 
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/SINAN/SifAquirida_2010_2017_Long.csv', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,10,19,20,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


O primeiro passo é descobrir filtrar as notificações feitas no município

In [4]:
# criando novo dataframe
filtered_df = df #.query('id_municip == 240810') # Ignorado por enquanto

### Limpeza

São retiradas colunas que não serão úteis

In [5]:
# null columns
nulls = ['in_vincula', 'id_ocupa_n', 'codisinf', 'co_usucad','tpuninot']

# colunas cujos valores repetem ou não são uteis
ignorable = [ 'nu_idade_ant_string',  'id_agravo', 'copaisinf',
             'tp_sistema', 'nobaiinf', 'id_pais', 'dt_diagnostico',
             
            # não sao uteis ( id, nome, data de nascimentos)
             'ctrlsifan',
             # muitos valores nulos
             'co_usualt',
            ]

labels = nulls + ignorable
cleaned_df = filtered_df.drop(labels=labels, axis='columns')

In [6]:
cleaned_df.replace('\s+', np.nan, regex=True, inplace=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344358 entries, 0 to 344357
Data columns (total 33 columns):
dt_notific    344355 non-null object
sem_not       344355 non-null float64
nu_ano        342531 non-null float64
anodiag       342531 non-null float64
sg_uf_not     342530 non-null object
id_municip    342531 non-null float64
id_regiona    313840 non-null object
id_unidade    342523 non-null object
UFRES         342531 non-null float64
id_mn_resi    342531 non-null float64
dt_diag       342531 non-null object
sem_pri       342531 non-null float64
dt_nasc       334125 non-null object
nu_idade_n    342531 non-null float64
FXETARIA      342531 non-null float64
SEXO          342531 non-null float64
cs_gestant    342524 non-null object
cs_raca       335034 non-null object
cs_escol_n    316785 non-null object
cs_zona       331169 non-null object
dt_invest     243189 non-null object
classi_fin    329125 non-null object
criterio      231979 non-null object
tpautocto     232044 non-nul

In [7]:
# rearranja dt_obito
cleaned_df['obito'] = cleaned_df.dt_obito.notna()
cleaned_df.drop(labels=['dt_obito'], axis='columns', inplace=True)

cleaned_df.obito

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
344328    False
344329    False
344330    False
344331    False
344332    False
344333    False
344334    False
344335    False
344336    False
344337    False
344338    False
344339    False
344340    False
344341    False
344342    False
344343    False
344344    False
344345    False
344346    False
344347    False
344348    False
344349    False
344350    False
344351    False
344352    False
344353    False
344354    False
344355    False
344356    False
344357    False
Name: obito, Length: 344

Como escolaridade é um dado importante, as linhas que não contém esse dado serão desconsiderados. Os dados de classificação final também são importantes, mas os dados sem classificacao serão prenchidos com 9, que é valor de "ignorado" no formulário.

In [8]:
cleaned_df.dropna(subset=['cs_escol_n'], axis='rows', inplace=True)

# verifica resultado
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316785 entries, 0 to 344357
Data columns (total 33 columns):
dt_notific    316785 non-null object
sem_not       316785 non-null float64
nu_ano        316785 non-null float64
anodiag       316785 non-null float64
sg_uf_not     316784 non-null object
id_municip    316785 non-null float64
id_regiona    290971 non-null object
id_unidade    316778 non-null object
UFRES         316785 non-null float64
id_mn_resi    316785 non-null float64
dt_diag       316785 non-null object
sem_pri       316785 non-null float64
dt_nasc       309481 non-null object
nu_idade_n    316785 non-null float64
FXETARIA      316785 non-null float64
SEXO          316785 non-null float64
cs_gestant    316781 non-null object
cs_raca       315950 non-null object
cs_escol_n    316785 non-null object
cs_zona       307924 non-null object
dt_invest     225868 non-null object
classi_fin    304551 non-null object
criterio      216371 non-null object
tpautocto     216454 non-nul

In [9]:
not_categorical = ['cs_gestant', 'cs_raca', 'cs_escol_n', 'cs_zona', 
          'tpautocto', 'classi_fin', 'criterio', 'evolucao', 
          'doenca_tra', 'sg_uf_not', 'id_regiona', 'id_unidade', 'coufinf']

cleaned_df.loc[:,not_categorical] = cleaned_df.loc[:,not_categorical].fillna( value=9)

cleaned_df.loc[:,not_categorical] = cleaned_df.loc[:,not_categorical].astype('float64')

cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316785 entries, 0 to 344357
Data columns (total 33 columns):
dt_notific    316785 non-null object
sem_not       316785 non-null float64
nu_ano        316785 non-null float64
anodiag       316785 non-null float64
sg_uf_not     316785 non-null float64
id_municip    316785 non-null float64
id_regiona    316785 non-null float64
id_unidade    316785 non-null float64
UFRES         316785 non-null float64
id_mn_resi    316785 non-null float64
dt_diag       316785 non-null object
sem_pri       316785 non-null float64
dt_nasc       309481 non-null object
nu_idade_n    316785 non-null float64
FXETARIA      316785 non-null float64
SEXO          316785 non-null float64
cs_gestant    316785 non-null float64
cs_raca       316785 non-null float64
cs_escol_n    316785 non-null float64
cs_zona       316785 non-null float64
dt_invest     225868 non-null object
classi_fin    316785 non-null float64
criterio      316785 non-null float64
tpautocto     31678

In [10]:
# Converte campos de data

cleaned_df.dt_notific = pd.to_datetime(cleaned_df.dt_notific,
                                           format='%m/%d/%Y')
cleaned_df.dt_diag = pd.to_datetime(cleaned_df.dt_diag,
                                           format='%m/%d/%Y')
cleaned_df.dt_encerra = pd.to_datetime(cleaned_df.dt_encerra,
                                           format='%m/%d/%Y')
cleaned_df.dt_invest.replace('216', '2016', regex=True,inplace=True)
cleaned_df.dt_invest = pd.to_datetime(cleaned_df.dt_invest,
                                           format='%m/%d/%Y')
cleaned_df.dt_nasc = pd.to_datetime(cleaned_df.dt_nasc,
                                           format='%m/%d/%Y')

cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316785 entries, 0 to 344357
Data columns (total 33 columns):
dt_notific    316785 non-null datetime64[ns]
sem_not       316785 non-null float64
nu_ano        316785 non-null float64
anodiag       316785 non-null float64
sg_uf_not     316785 non-null float64
id_municip    316785 non-null float64
id_regiona    316785 non-null float64
id_unidade    316785 non-null float64
UFRES         316785 non-null float64
id_mn_resi    316785 non-null float64
dt_diag       316785 non-null datetime64[ns]
sem_pri       316785 non-null float64
dt_nasc       309481 non-null datetime64[ns]
nu_idade_n    316785 non-null float64
FXETARIA      316785 non-null float64
SEXO          316785 non-null float64
cs_gestant    316785 non-null float64
cs_raca       316785 non-null float64
cs_escol_n    316785 non-null float64
cs_zona       316785 non-null float64
dt_invest     225868 non-null datetime64[ns]
classi_fin    316785 non-null float64
criterio      316785 non-

In [11]:
# corrige idade
cleaned_df['idade'] = (cleaned_df.dt_diag - cleaned_df.dt_nasc).dt.days //365

cleaned_df.drop(labels=['dt_nasc'], axis='columns', inplace=True)

cleaned_df['idade']

0         16.0
1         44.0
2         24.0
3         51.0
4          NaN
5         55.0
6         23.0
7         15.0
8         57.0
9         40.0
10        56.0
11        48.0
12        45.0
13        27.0
14        28.0
15        22.0
16        58.0
17        57.0
18        20.0
19        46.0
20        18.0
21        44.0
22        37.0
23        33.0
24        45.0
25        41.0
27        26.0
28        35.0
29        35.0
30        34.0
          ... 
344325    21.0
344327    37.0
344328    23.0
344329    17.0
344330    25.0
344331    56.0
344332    42.0
344333    27.0
344334    52.0
344335    24.0
344336    57.0
344337    46.0
344338    64.0
344339    26.0
344340    35.0
344341    20.0
344343    62.0
344344    61.0
344345    58.0
344346    34.0
344348    31.0
344349    27.0
344350    67.0
344351    50.0
344352    39.0
344353    19.0
344354    47.0
344355    33.0
344356    30.0
344357    37.0
Name: idade, Length: 316785, dtype: float64

In [12]:
cleaned_df.loc[:, ['dt_diag', 'nu_idade', 'idade']]

,dt_diag,nu_idade,idade
0,2010-01-01,16.0,16.0
1,2010-01-01,44.0,44.0
2,2010-01-01,24.0,24.0
3,2010-01-01,51.0,51.0
4,2010-01-01,41.0,NaN
5,2010-01-01,55.0,55.0
6,2010-01-01,23.0,23.0
7,2010-01-01,15.0,15.0
8,2010-01-01,57.0,57.0
9,2010-01-01,40.0,40.0


In [13]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316785 entries, 0 to 344357
Data columns (total 33 columns):
dt_notific    316785 non-null datetime64[ns]
sem_not       316785 non-null float64
nu_ano        316785 non-null float64
anodiag       316785 non-null float64
sg_uf_not     316785 non-null float64
id_municip    316785 non-null float64
id_regiona    316785 non-null float64
id_unidade    316785 non-null float64
UFRES         316785 non-null float64
id_mn_resi    316785 non-null float64
dt_diag       316785 non-null datetime64[ns]
sem_pri       316785 non-null float64
nu_idade_n    316785 non-null float64
FXETARIA      316785 non-null float64
SEXO          316785 non-null float64
cs_gestant    316785 non-null float64
cs_raca       316785 non-null float64
cs_escol_n    316785 non-null float64
cs_zona       316785 non-null float64
dt_invest     225868 non-null datetime64[ns]
classi_fin    316785 non-null float64
criterio      316785 non-null float64
tpautocto     316785 non-null fl

In [15]:
cleaned_df.sg_uf_not.value_counts().head()

35.0    134025
43.0     35396
31.0     23338
33.0     20285
41.0     16932
Name: sg_uf_not, dtype: int64

In [15]:
cleaned_df.to_csv('../../data/adquirida.csv', sep=';')